###安裝套件

In [77]:
!pip install transformers

###載入資料

資料來源：https://www.kaggle.com/competitions/stock-market-prediction-and-sentimental-analysis/data

In [78]:
!unzip "/content/drive/MyDrive/Colab Notebooks/金融大數據/stock-market-prediction-and-sentimental-analysis.zip"

Archive:  /content/drive/MyDrive/Colab Notebooks/金融大數據/stock-market-prediction-and-sentimental-analysis.zip
replace Combined_News_DJIA(train).csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace DJIA_table(train).csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace RedditNews(train).csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TEST_Redit_news.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Test_Combined_News.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Test_DJIA_Table.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


###處理新聞資料

In [79]:
import pandas as pd

news_df = pd.read_csv("/content/RedditNews(train).csv")
news_df.head()

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...


In [80]:
import re

# 定義函數移除標點符號
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', ' ', text)

# 移除 "News" 欄位中的標點符號
news_df['News'] = news_df['News'].apply(remove_punctuation)

news_df['News'] = news_df['News'].str.lower()

news_df.head()

,Date,News
0,2016-07-01,a 117 year old woman in mexico city finally re...
1,2016-07-01,imf chief backs athens as permanent olympic host
2,2016-07-01,the president of france says if brexit won so...
3,2016-07-01,british man who must give police 24 hours not...
4,2016-07-01,100 nobel laureates urge greenpeace to stop o...


In [81]:
news_dict = news_df.groupby('Date')['News'].apply(list).to_dict()
news_dict

{'2008-06-08': ['b nim chimpsky  the tragedy of the chimp who thought he was a boy  and proved that humans were not humane  ',
  'b canada  beware slippery slope  to censorship  hearing told  ',
  'b eu vice president luisa morgantini and the irish nobel laureate  mairead corrigan  have been tear gased and injured by the idf while attending the  international conference on non violent resistance  ',
  'b israeli minister  israel will attack iran if it doesn t abandon its nuclear program ',
  'b albino killings in tanzania  at least 19 albinos  including several young children  have been killed in tanzania in the past year   video   ',
  'b chiapas  army occupies zapatista communities in  anti drug  ops ',
  'b polar bear swims 200 miles  is shot dead upon arrival ',
  'b news is a contraband item in pakistan now  and it is being sold on the black market  ',
  'b albinos  long shunned  face threat in tanzania where witch doctors are now marketing albino skin  bones and hair as ingredien

In [82]:
from transformers import AutoTokenizer, AutoModel
import torch

# 初始化 BERT Tokenizer 和 Model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")

bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [83]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
device

device(type='cuda')

In [84]:
date_embeddings = {}

for date, news_list in news_dict.items():
    # 合併當天的所有新聞
    combined_text = " ".join(news_list)

    # 將文本 Tokenize 並輸入到 BERT
    inputs = tokenizer(combined_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = bert_model(**inputs)
    print(f'{date} done.')

    # 提取 [CLS] Token 的嵌入作為整體表示
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).detach().cpu().numpy()

    # 保存嵌入到字典
    date_embeddings[date] = cls_embedding

2008-06-08 done.
2008-06-09 done.
2008-06-10 done.
2008-06-11 done.
2008-06-12 done.
2008-06-13 done.
2008-06-14 done.
2008-06-15 done.
2008-06-16 done.
2008-06-17 done.
2008-06-18 done.
2008-06-19 done.
2008-06-20 done.
2008-06-21 done.
2008-06-22 done.
2008-06-23 done.
2008-06-24 done.
2008-06-25 done.
2008-06-26 done.
2008-06-27 done.
2008-06-28 done.
2008-06-29 done.
2008-06-30 done.
2008-07-01 done.
2008-07-02 done.
2008-07-03 done.
2008-07-04 done.
2008-07-05 done.
2008-07-06 done.
2008-07-07 done.
2008-07-08 done.
2008-07-09 done.
2008-07-10 done.
2008-07-11 done.
2008-07-12 done.
2008-07-13 done.
2008-07-14 done.
2008-07-15 done.
2008-07-16 done.
2008-07-17 done.
2008-07-18 done.
2008-07-19 done.
2008-07-20 done.
2008-07-21 done.
2008-07-22 done.
2008-07-23 done.
2008-07-24 done.
2008-07-25 done.
2008-07-26 done.
2008-07-27 done.
2008-07-28 done.
2008-07-29 done.
2008-07-30 done.
2008-07-31 done.
2008-08-01 done.
2008-08-02 done.
2008-08-03 done.
2008-08-04 done.
2008-08-05 don

In [85]:
embedding_df = pd.DataFrame(date_embeddings).T
print(embedding_df)
embedding_df.columns = [f"Embedding_{i}" for i in range(embedding_df.shape[1])]
embedding_df['Date'] = embedding_df.index
embedding_df['Date'] = pd.to_datetime(embedding_df['Date'], format='%Y-%m-%d')
embedding_df.head()

                 0         1         2         3         4         5    \
2008-06-08 -0.476054  0.064374  0.215406 -0.040232 -0.282752 -0.106417   
2008-06-09 -0.564087  0.134622  0.268847 -0.237865 -0.434964 -0.279899   
2008-06-10 -0.228911  0.250396  0.089964  0.074538 -0.359633 -0.114905   
2008-06-11 -0.447869  0.254780  0.137295  0.250122 -0.317880 -0.029660   
2008-06-12 -0.454670  0.240624  0.089035 -0.148121 -0.711615 -0.000221   
...              ...       ...       ...       ...       ...       ...   
2016-06-27 -0.581780  0.323513  0.295291 -0.181167 -0.603238  0.198646   
2016-06-28 -0.403103  0.408799  0.263755 -0.050978 -0.195771  0.256436   
2016-06-29 -0.300584  0.203734  0.103020 -0.081582 -0.579146  0.106429   
2016-06-30 -0.484202  0.089565  0.224236  0.331181 -0.501259  0.012853   
2016-07-01 -0.465222  0.130904  0.340813  0.253206 -0.331281  0.013035   

                 6         7         8         9    ...       758       759  \
2008-06-08  0.418539  0.397045 -

,Embedding_0,Embedding_1,Embedding_2,Embedding_3,Embedding_4,Embedding_5,Embedding_6,Embedding_7,Embedding_8,Embedding_9,...,Embedding_759,Embedding_760,Embedding_761,Embedding_762,Embedding_763,Embedding_764,Embedding_765,Embedding_766,Embedding_767,Date
2008-06-08,-0.476054,0.064374,0.215406,-0.040232,-0.282752,-0.106417,0.418539,0.397045,-0.072241,0.054044,...,-0.578743,-0.301886,-0.081890,0.419775,0.215962,-0.225385,-0.454791,0.532129,0.135489,2008-06-08
2008-06-09,-0.564087,0.134622,0.268847,-0.237865,-0.434964,-0.279899,0.402961,0.423638,0.022004,-0.014428,...,-0.515940,-0.146966,-0.049449,0.736530,0.353016,-0.147685,-0.386908,0.326534,0.183220,2008-06-09
2008-06-10,-0.228911,0.250396,0.089964,0.074538,-0.359633,-0.114905,0.623977,0.327524,0.003973,-0.032990,...,-0.568226,-0.566934,-0.045105,0.679314,0.161246,-0.120224,-0.252589,0.639550,0.174145,2008-06-10
2008-06-11,-0.447869,0.254780,0.137295,0.250122,-0.317880,-0.029660,0.658889,0.281889,-0.407415,-0.023638,...,-0.221866,-0.262341,-0.118754,0.552238,0.063628,-0.323020,-0.449656,0.390991,-0.007618,2008-06-11
2008-06-12,-0.454670,0.240624,0.089035,-0.148121,-0.711615,-0.000221,0.445988,0.589419,-0.289855,0.040114,...,-0.410221,-0.235234,0.034233,0.587021,-0.032611,-0.121900,-0.314988,0.778452,0.145275,2008-06-12


###處理股價資料

In [86]:
stock_df = pd.read_csv("/content/DJIA_table(train).csv")
stock_df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,31-12-2015,17590.66016,17590.66016,17421.16016,17425.02930,93690000,17425.02930
1,30-12-2015,17711.93945,17714.13086,17588.86914,17603.86914,59760000,17603.86914
2,29-12-2015,17547.36914,17750.01953,17547.36914,17720.98047,69860000,17720.98047
3,28-12-2015,17535.66016,17536.90039,17437.33984,17528.26953,59770000,17528.26953
4,24-12-2015,17593.25977,17606.33984,17543.94922,17552.16992,40350000,17552.16992


In [87]:
# 將 'Date' 欄位轉換為日期格式
stock_df['Date'] = pd.to_datetime(stock_df['Date'], dayfirst=True)

stock_df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-31,17590.66016,17590.66016,17421.16016,17425.02930,93690000,17425.02930
1,2015-12-30,17711.93945,17714.13086,17588.86914,17603.86914,59760000,17603.86914
2,2015-12-29,17547.36914,17750.01953,17547.36914,17720.98047,69860000,17720.98047
3,2015-12-28,17535.66016,17536.90039,17437.33984,17528.26953,59770000,17528.26953
4,2015-12-24,17593.25977,17606.33984,17543.94922,17552.16992,40350000,17552.16992


In [88]:
# 計算 SMA
def calculate_SMA(data, window):
    return data['Close'].rolling(window=window).mean()

# 計算技術指標
stock_df['SMA_8'] = calculate_SMA(stock_df, 8)
stock_df['SMA_20'] = calculate_SMA(stock_df, 20)

stock_df

,Date,Open,High,Low,Close,Volume,Adj Close,SMA_8,SMA_20
0,2015-12-31,17590.66016,17590.66016,17421.16016,17425.02930,93690000,17425.02930,NaN,NaN
1,2015-12-30,17711.93945,17714.13086,17588.86914,17603.86914,59760000,17603.86914,NaN,NaN
2,2015-12-29,17547.36914,17750.01953,17547.36914,17720.98047,69860000,17720.98047,NaN,NaN
3,2015-12-28,17535.66016,17536.90039,17437.33984,17528.26953,59770000,17528.26953,NaN,NaN
4,2015-12-24,17593.25977,17606.33984,17543.94922,17552.16992,40350000,17552.16992,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1858,2008-08-14,11532.07031,11718.28027,11450.88965,11615.92969,159790000,11615.92969,11495.714845,11452.166455
1859,2008-08-13,11632.80957,11633.78027,11453.33984,11532.95996,182550000,11532.95996,11514.053590,11457.128956
1860,2008-08-12,11781.70020,11782.34961,11601.51953,11642.46973,173590000,11642.46973,11515.854860,11475.806446
1861,2008-08-11,11729.66992,11867.11035,11675.53027,11782.34961,183190000,11782.34961,11559.872316,11503.387403


In [89]:
# 處理股價資料，計算漲跌標籤
stock_df['Price Change'] = stock_df['Close'] - stock_df['Close'].shift(-1)
stock_df['Label'] = stock_df['Price Change'].apply(lambda x: 1 if x > 0 else 0)
stock_df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,SMA_8,SMA_20,Price Change,Label
0,2015-12-31,17590.66016,17590.66016,17421.16016,17425.02930,93690000,17425.02930,NaN,NaN,-178.83984,0
1,2015-12-30,17711.93945,17714.13086,17588.86914,17603.86914,59760000,17603.86914,NaN,NaN,-117.11133,0
2,2015-12-29,17547.36914,17750.01953,17547.36914,17720.98047,69860000,17720.98047,NaN,NaN,192.71094,1
3,2015-12-28,17535.66016,17536.90039,17437.33984,17528.26953,59770000,17528.26953,NaN,NaN,-23.90039,0
4,2015-12-24,17593.25977,17606.33984,17543.94922,17552.16992,40350000,17552.16992,NaN,NaN,-50.43946,0


###合併資料

In [90]:
stock = stock_df[['Date', 'Close', 'SMA_8', "SMA_20", 'Label']]
stock.head()

,Date,Close,SMA_8,SMA_20,Label
0,2015-12-31,17425.02930,NaN,NaN,0
1,2015-12-30,17603.86914,NaN,NaN,0
2,2015-12-29,17720.98047,NaN,NaN,1
3,2015-12-28,17528.26953,NaN,NaN,0
4,2015-12-24,17552.16992,NaN,NaN,0


In [91]:
print(embedding_df['Date'].dtype)
print(stock['Date'].dtype)

datetime64[ns]
datetime64[ns]


In [92]:
combine_df = pd.merge(embedding_df,stock,  on='Date', how='inner')
combine_df.set_index('Date', inplace=True)
combine_df.head()

,Embedding_0,Embedding_1,Embedding_2,Embedding_3,Embedding_4,Embedding_5,Embedding_6,Embedding_7,Embedding_8,Embedding_9,...,Embedding_762,Embedding_763,Embedding_764,Embedding_765,Embedding_766,Embedding_767,Close,SMA_8,SMA_20,Label
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-08,-0.522073,0.025530,-0.142223,0.044214,-0.641205,-0.234618,0.367696,0.798160,-0.019327,-0.028395,...,0.383625,-0.029097,-0.107272,-0.482263,0.364909,-0.000314,11734.32031,11599.483644,11514.566407,0
2008-08-11,-0.194733,0.087088,0.030497,0.228891,-0.503841,-0.279069,0.381677,0.644309,-0.108829,-0.127970,...,0.782425,0.349535,-0.071546,-0.422619,0.477981,0.192547,11782.34961,11559.872316,11503.387403,1
2008-08-12,-0.695120,0.246100,-0.002264,0.270665,-0.980139,-0.152320,0.681986,0.293841,-0.091851,-0.052153,...,0.580897,0.502981,-0.041480,-0.491304,0.923219,0.226407,11642.46973,11515.854860,11475.806446,0
2008-08-13,-0.585735,0.093470,0.095165,0.052195,-0.638326,-0.173666,0.393801,0.516593,-0.344405,0.106823,...,0.591231,0.103198,-0.003542,-0.382727,0.636118,0.030511,11532.95996,11514.053590,11457.128956,0
2008-08-14,-0.457878,0.027801,-0.087460,0.108675,-0.674753,-0.133750,0.476994,0.463141,0.163585,-0.061879,...,0.551956,0.117465,-0.232179,-0.363722,0.739028,-0.151286,11615.92969,11495.714845,11452.166455,1


###前期融合

In [93]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical


# 特徵和標籤
features = combine_df.iloc[:, :-1]  # 所有特徵（去掉 Label）
labels = combine_df['Label']        # 漲跌標籤

# 標準化
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# 構建時間窗口
def create_sequences(data, labels, time_steps=5):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 5  # 過去 5 天的數據預測
X, y = create_sequences(features_scaled, labels.values, time_steps)

# 將標籤轉為分類格式
y = to_categorical(y)


In [94]:
from tensorflow.keras import layers, models

# 定義 LSTM 模型
model = models.Sequential([
    layers.LSTM(64, activation='tanh', return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    layers.Dropout(0.3),
    layers.LSTM(32, activation='tanh'),
    layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    layers.Dense(2, activation='softmax')  # 二分類輸出
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 5, 64)               │         214,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 5, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 226,994 (886.70 KB)

 Trainable params: 226,994 (886.70 KB)

 Non-trainable params: 0 (0.00 B)

In [95]:
from sklearn.model_selection import train_test_split

# 拆分訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# 訓練模型
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test)
)


Epoch 1/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5108 - loss: 0.6977 - val_accuracy: 0.5081 - val_loss: nan
Epoch 2/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5981 - loss: 0.6683 - val_accuracy: 0.5108 - val_loss: nan
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6431 - loss: 0.6472 - val_accuracy: 0.5054 - val_loss: nan
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7111 - loss: 0.5907 - val_accuracy: 0.4866 - val_loss: nan
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7234 - loss: 0.5511 - val_accuracy: 0.4919 - val_loss: nan
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7469 - loss: 0.5202 - val_accuracy: 0.4919 - val_loss: nan
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7575 - loss: 0.4926 - val_accuracy: 0.5054 - val_loss: nan
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8073 - loss: 0.4277 - val_accuracy: 0.4785 - val_loss: nan
Epoch 9/2

In [96]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5166 - loss: nan    
Test Accuracy: 0.51


In [98]:
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)  # 將概率轉為 0 或 1
print(predicted_labels)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0
 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1
 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1
 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1
 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


###中期融合

In [100]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, Input
from tensorflow.keras.utils import to_categorical

# 特徵和標籤
features = combine_df.iloc[:, :-1]  # 所有特徵（去掉 Label）
labels = combine_df['Label']        # 漲跌標籤
# 分離嵌入向量和技術指標
embedding_features = features.iloc[:, :768].values  # BERT 嵌入向量
technical_features = features[['Close', 'SMA_8', 'SMA_20']].values  # 技術指標
labels = to_categorical(labels)  # 將漲跌標籤轉為 one-hot

# 標準化技術指標
scaler = StandardScaler()
technical_features_scaled = scaler.fit_transform(technical_features)

# 創建時間序列
def create_sequences(data, time_steps=5):
    sequences = []
    for i in range(len(data) - time_steps):
        sequences.append(data[i:i+time_steps])
    return np.array(sequences)

time_steps = 5
X_embeddings = create_sequences(embedding_features, time_steps)
X_technical = create_sequences(technical_features_scaled, time_steps)
y = labels[time_steps:]  # 標籤對應的起始位置對齊時間序列

# 確認形狀
print(f"嵌入特徵形狀: {X_embeddings.shape}")
print(f"技術指標形狀: {X_technical.shape}")
print(f"標籤形狀: {y.shape}")

嵌入特徵形狀: (1858, 5, 768)
技術指標形狀: (1858, 5, 3)
標籤形狀: (1858, 2)


In [103]:
# 切分訓練集和測試集(80%訓練，20%測試)
split_index = int(len(X_technical) * 0.8)

# 股價序列
X_technical_train = X_technical[:split_index]
X_technical_test = X_technical[split_index:]

# 新聞序列
X_embeddings_train = X_embeddings[:split_index]
X_embeddings_test = X_embeddings[split_index:]

# 目標值（0:跌, 1:漲）
y_train = y[:split_index]
y_test = y[split_index:]

In [101]:
# 模型結構設計
# 嵌入向量分支
embedding_input = Input(shape=(X_embeddings.shape[1], X_embeddings.shape[2]), name='embedding_input')
embedding_branch = layers.LSTM(64, activation='tanh', return_sequences=True)(embedding_input)
embedding_branch = layers.Dropout(0.3)(embedding_branch)
embedding_branch = layers.LSTM(32, activation='tanh')(embedding_branch)

# 技術指標分支
technical_input = Input(shape=(X_technical.shape[1], X_technical.shape[2]), name='technical_input')
technical_branch = layers.Dense(32, activation='relu')(technical_input)
technical_branch = layers.Flatten()(technical_branch)

# 中期融合
merged = layers.Concatenate()([embedding_branch, technical_branch])
merged = layers.Dense(64, activation='relu')(merged)
merged = layers.Dropout(0.3)(merged)
output = layers.Dense(2, activation='softmax', name='output')(merged)

# 建立模型
model = models.Model(inputs=[embedding_input, technical_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ embedding_input           │ (None, 5, 768)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ (None, 5, 64)          │        213,248 │ embedding_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ technical_input           │ (None, 5, 3)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 5, 64)          │              0 │ lstm_2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 5, 32)          │            128 │ technical_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ (None, 32)             │         12,416 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 160)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 192)            │              0 │ lstm_3[0][0],          │
│                           │                        │                │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 64)             │         12,352 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 64)             │              0 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 2)              │            130 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 238,274 (930.76 KB)

 Trainable params: 238,274 (930.76 KB)

 Non-trainable params: 0 (0.00 B)

In [104]:
# 訓練模型
history = model.fit(
    [X_embeddings_train, X_technical_train], y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6465 - loss: 0.6372 - val_accuracy: 0.6141 - val_loss: 0.6531
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6253 - loss: 0.6397 - val_accuracy: 0.6376 - val_loss: 0.6441
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6257 - loss: 0.6400 - val_accuracy: 0.6342 - val_loss: 0.6423
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6401 - loss: 0.6246 - val_accuracy: 0.6309 - val_loss: 0.6401
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6198 - loss: 0.6368 - val_accuracy: 0.6309 - val_loss: 0.6411
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6832 - loss: 0.6120 - val_accuracy: 0.6409 - val_loss: 0.6362
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6672 - loss: 0.6061 - val_accuracy: 0.6510 - val_loss: 0.6350
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6747 - loss: 0.6159 - val_accuracy: 0.6342 - v

In [107]:
# 使用測試集評估
test_loss, test_accuracy = model.evaluate([X_embeddings_test, X_technical_test], y_test, batch_size=32)
print(f"測試集準確率: {test_accuracy}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5654 - loss: nan   
測試集準確率: 0.5940860509872437


###後期融合

In [109]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical


# 特徵和標籤
features = combine_df.iloc[:, :-1]  # 所有特徵（去掉 Label）
labels = combine_df['Label']        # 漲跌標籤

# 分成訓練集 (80%) 和測試集 (20%)
train_size = int(len(features) * 0.8)
X_train, X_test = features.iloc[:train_size], features.iloc[train_size:]
y_train, y_test = labels.iloc[:train_size], labels.iloc[train_size:]

print("訓練集資料筆數:", len(X_train))
print("測試集資料筆數:", len(X_test))

訓練集資料筆數: 1490
測試集資料筆數: 373


In [110]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 模型 A: LSTM
def build_model_a(input_shape):
    model = Sequential([
        LSTM(128, activation='tanh', return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(64, activation='tanh'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  # 二元分類
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 定義輸入形狀
input_shape_a = (768, 1)  # BERT 向量有 768 個特徵
model_a = build_model_a(input_shape_a)
model_a.summary()

# 提取 BERT 嵌入向量
X_train_bert = X_train.filter(like='Embedding_')
X_test_bert = X_test.filter(like='Embedding_')

# 訓練模型 A
model_a.fit(X_train_bert, y_train, epochs=10, batch_size=32, validation_split=0.2)
lstm_predictions_train = model_a.predict(X_train_bert).flatten()
lstm_predictions_test = model_a.predict(X_test_bert).flatten()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 768, 128)            │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 768, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 118,081 (461.25 KB)

 Trainable params: 118,081 (461.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - accuracy: 0.5165 - loss: 0.6931 - val_accuracy: 0.5470 - val_loss: 0.6887
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.5488 - loss: 0.6893 - val_accuracy: 0.5470 - val_loss: 0.6891
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.5353 - loss: 0.6908 - val_accuracy: 0.5470 - val_loss: 0.6887
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.5344 - loss: 0.6944 - val_accuracy: 0.5470 - val_loss: 0.6888
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.5256 - loss: 0.6923 - val_accuracy: 0.5470 - val_loss: 0.6888
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.5274 - loss: 0.6933 - val_accuracy: 0.5470 - val_loss: 0.6886
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.5305 - loss: 0.6923 - val_accuracy: 0.5470 - val_loss: 0.6887
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.5453 - loss: 0.6899 - val_accuracy: 0.5470 - 

In [111]:
def build_model_b(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  # 二元分類
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
# 提取技術指標
X_train_tech = X_train[['Close', 'SMA_8', 'SMA_20']]
X_test_tech = X_test[['Close', 'SMA_8', 'SMA_20']]

# 定義輸入形狀
input_shape_b = (X_train_tech.shape[1],)  # 技術指標的特徵數量
model_b = build_model_b(input_shape_b)
model_b.summary()

# 訓練模型 B
model_b.fit(X_train_tech, y_train, epochs=10, batch_size=32, validation_split=0.2)
tech_predictions_train = model_b.predict(X_train_tech).flatten()
tech_predictions_test = model_b.predict(X_test_tech).flatten()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 64)                  │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,369 (9.25 KB)

 Trainable params: 2,369 (9.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.4873 - loss: 1041.2697 - val_accuracy: 0.4530 - val_loss: 84.7656
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5214 - loss: 241.0284 - val_accuracy: 0.4530 - val_loss: 17.8750
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5053 - loss: 182.3567 - val_accuracy: 0.4530 - val_loss: 90.4404
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5248 - loss: 118.6496 - val_accuracy: 0.5470 - val_loss: 17.9422
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5238 - loss: 86.8174 - val_accuracy: 0.5470 - val_loss: 32.6563
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5326 - loss: 52.8798 - val_accuracy: 0.4530 - val_loss: 16.7706
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4793 - loss: 39.8675 - val_accuracy: 0.5470 - val_loss: 26.8745
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5240 - loss: 25.1378 - val_accurac

In [112]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 合併兩個模型的預測
X_train_fused = np.column_stack((lstm_predictions_train, tech_predictions_train))
X_test_fused = np.column_stack((lstm_predictions_test, tech_predictions_test))

# 訓練最終分類器
final_model = RandomForestClassifier(random_state=42)
final_model.fit(X_train_fused, y_train)

RandomForestClassifier(random_state=42)

In [113]:
# 評估結果
train_preds = final_model.predict(X_train_fused)
test_preds = final_model.predict(X_test_fused)
print("訓練集準確率:", accuracy_score(y_train, train_preds))
print("測試集準確率:", accuracy_score(y_test, test_preds))

訓練集準確率: 0.763758389261745
測試集準確率: 0.48257372654155495
